In [1]:
from dataclasses import dataclass
from pathlib import Path

import pandas as pd

from mmvae_hub.experiment_vis.utils import *
from mmvae_hub.base.utils.MongoDB import MongoDatabase
from mmvae_hub.base.utils.flags_utils import get_config_path, BaseFlagsSetup
from mmvae_hub.base.utils.utils import dict2pyobject, json2dict

[09:38:06 CEST] Log file: /usr/local/var/logs/mmvae_hub/UTC-20210521-073806.log


In [2]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

In [3]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  36 experiments in the database.


In [4]:
df = make_experiments_dataframe(experiments)

In [5]:
show_cols = ['weighted_mixture','score', 'method', 'end_epoch', 'class_dim', 'beta', '_id', 'score_lr', 'score_gen','score_prd', 'version']
df[show_cols].sort_values(by=['score'], ascending=False)

,weighted_mixture,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen,score_prd,version
0,NaN,15.368883,joint_elbo,299,64.0,2.5,polymnist_joint_elbo_2021_05_04_09_21_34_487039,5.252283,10.116600,0.000000,NaN
23,0.0,14.428333,joint_elbo,99,512.0,2.5,polymnist_joint_elbo_2021_05_19_10_59_36_964676,5.514483,8.913850,0.288514,0.0.4-dev
18,NaN,13.342450,moe,299,512.0,2.5,polymnist_moe_2021_05_16_08_11_11_173474,4.227050,9.115400,0.433640,0.0.3-dev
4,NaN,11.739267,joint_elbo,299,64.0,0.5,polymnist_joint_elbo_2021_05_08_05_21_05_683057,3.932350,7.806917,0.611546,0.0.1-dev
15,NaN,11.271117,planar_mixture_5,299,512.0,2.5,polymnist_planar_mixture_2021_05_13_15_20_43_7...,1.477533,9.793583,2.595059,0.0.2-dev
13,NaN,11.132117,planar_mixture_5,299,512.0,2.5,polymnist_planar_mixture_2021_05_12_11_38_58_1...,1.349800,9.782317,1.495992,0.0.2-dev
5,NaN,11.092400,moe,299,128.0,0.5,polymnist_moe_2021_05_09_01_46_10_858142,3.572933,7.519467,0.332367,0.0.1-dev
2,NaN,9.968633,joint_elbo,299,32.0,0.5,polymnist_joint_elbo_2021_05_06_12_59_58_785343,3.239317,6.729317,0.312330,0.0.1-dev
28,0.0,8.978700,pfom_0,99,512.0,2.5,polymnist_pfom_2021_05_20_16_48_25_478210,1.766483,7.212217,0.187531,0.0.5-dev
3,NaN,7.705983,moe,299,64.0,0.5,polymnist_moe_2021_05_07_09_09_37_528857,2.489417,5.216567,0.192170,0.0.1-dev


# Explore all pfom experiments

In [6]:
df.loc[df['method'].str.startswith('pfom')][[*show_cols, 'expvis_url']]

,weighted_mixture,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen,score_prd,version,expvis_url
28,0.0,8.978700,pfom_0,99,512.0,2.5,polymnist_pfom_2021_05_20_16_48_25_478210,1.766483,7.212217,0.187531,0.0.5-dev,https://ppb.hendrikklug.xyz/9ba46d23db0a.pdf
29,1.0,2.836100,pfom_5,99,512.0,2.5,polymnist_pfom_2021_05_20_19_25_21_500572,0.699233,2.136867,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/ce456ef3a982.pdf
30,0.0,2.845933,pfom_5,99,512.0,2.5,polymnist_pfom_2021_05_20_20_27_06_214720,0.703383,2.142550,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/69e2e11db816.pdf


Generate samples with the model that has the highest score.

In [7]:
best_score_id = df[df.score == df.score.max()]._id.item()

print(best_score_id)
experiments_database = MongoDatabase(training=False, _id=best_score_id)
experiment_dict = experiments_database.get_experiment_dict()
config = json2dict(get_config_path())
experiment_flags = experiment_dict['flags']

flags = BaseFlagsSetup.set_paths_with_config(config, experiment_flags, is_dict=True)

experiment_flags['use_clf'] = False
experiment_flags['device'] = 'cpu'
experiment_flags['weighted_mixture'] = False
experiment_flags['save_figure'] = False



polymnist_joint_elbo_2021_05_04_09_21_34_487039


In [8]:
show_generated_figs(flags=dict2pyobject(experiment_flags))

UnboundLocalError: local variable 'dp' referenced before assignment